In [4]:
!pip install -q # xgboost seaborn


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

In [6]:
train_data = './Training_dataset.csv'
test_data  = './New_Validation_Dataset.csv'

In [7]:
def getData(path):
    """
    Функция для извлечения данных из текстового файла по заданному пути.

    Данные должны быть организованы в файле таким образом, что каждая строка содержит метку и последовательность,
    разделённые запятой. Первая строка файла, содержащая слово 'Label', игнорируется как заголовок.

    Параметры:
    path : str
        Путь к файлу с данными для чтения.

    Возвращает:
    tuple
        Возвращает кортеж из двух списков: первый список содержит последовательности (sequence),
        а второй список содержит соответствующие метки (label).

    Пример использования:
    sequence, label = getData('path/to/your/data.txt')
    """
    sequence = []
    label = []

    with open(path) as f:
        for line in f:
            text_file = line.split(",")
            if text_file[0] == 'Label':
                continue
            label.append(text_file[0])
            sequence.append(text_file[1][:-1])

    return sequence, label

In [8]:
# Отделяет как метки, так и строку
x_train, y_train = getData(train_data)

print('Имеется',len(y_train),'количество доступных образцов ДНК.')

Имеется 38910 количество доступных образцов ДНК.


In [9]:
# Извлечение признаков дипептидов, в этом случае 780 признаков. Комбинация двух признаков с аминокислотной последовательностью
aminoacid_sequence = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

dipeptide_seq = []

for charone in aminoacid_sequence:
    for chartwo in aminoacid_sequence:
        char_to_add = charone+chartwo
        char_to_addback = chartwo + charone
        dipeptide_seq.append(char_to_add)
        if char_to_add == char_to_addback:
            continue
        dipeptide_seq.append(char_to_addback)

In [10]:
# Здесь для каждой последовательности извлекается 780 признаков дипептида с помощью расчетов дипептидов
def get_dipeptide_dictionary(string):
    """
    Функция для создания словаря дипептидов из заданной строки.

    Для каждой возможной пары символов в строке (включая смежные и через один символ) функция вычисляет
    относительную частоту каждой уникальной комбинации в строке. Относительная частота вычисляется как
    количество появлений комбинации, делённое на общую длину входной строки.

    Параметры:
    string : str
        Входная строка, для которой необходимо вычислить дипептидный словарь.

    Возвращает:
    dict
        Словарь, где ключами являются возможные комбинации двух символов (дипептиды), а значениями — их относительная частота
        во входной строке. Относительная частота рассчитывается как количество появлений дипептида, делённое на длину строки.

    Пример использования:
    dipeptide_dict = get_dipeptide_dictionary("ABCD")
    # Возвращает словарь вида {'AB': 0.25, 'BC': 0.25, 'CD': 0.25, 'AC': 0.25, 'BD': 0.25}
    """
    # Инициализация переменных
    size = len(string)  # Длина входной строки
    i = 0
    eachseq_list = []   # Список для хранения всех дипептидов

    # Генерация дипептидов из смежных символов
    while i < size - 1:
        add_seq = string[i] + string[i+1]
        eachseq_list.append(add_seq)
        i += 1

    # Генерация дипептидов с пропуском одного символа
    i = 0
    while i < size - 2:
        add_seq = string[i] + string[i+2]
        eachseq_list.append(add_seq)
        i += 1

    # Создание словаря для подсчёта вхождений каждого дипептида
    getdict = {}
    for item in eachseq_list:
        if item not in getdict:
            getdict[item] = 1
        else:
            getdict[item] += 1

    # Подсчёт относительной частоты каждого дипептида
    new_dict = {}
    for item, count in getdict.items():
        new_dict[item] = count / size

    return new_dict

In [11]:
# Эти две функции помогают нам создать вектор для каждой последовательности
def getlist_of_dictionary(x_train):
    """
    Преобразует список строк в список словарей дипептидных векторов.

    Каждый элемент списка представляет собой словарь, генерируемый функцией get_dipeptide_dictionary,
    для соответствующей строки в x_train.

    Параметры:
    x_train : list of str
        Список строк, для которых нужно сгенерировать дипептидные словари.

    Возвращает:
    list of dict
        Список словарей дипептидных векторов для каждой строки в x_train.

    Пример использования:
    list_dictionary = getlist_of_dictionary(["ABCD", "XYZ"])
    # Возвращает список словарей с дипептидными векторами для каждой строки.
    """
    list_dictionary = []  # Инициализация списка для хранения словарей

    # Перебор каждой строки в входном списке
    for each in x_train:
        dic = get_dipeptide_dictionary(each)
        list_dictionary.append(dic)

    return list_dictionary

def getvector(vector_dictionary, dipeptide_seq):
    """
    Преобразует список словарей дипептидных векторов в список векторов фиксированной длины.

    Каждый вектор представляет собой векторное представление одной последовательности, где каждый элемент вектора
    соответствует частоте одного дипептида из списка dipeptide_seq в этой последовательности.

    Параметры:
    vector_dictionary : list of dict
        Список словарей дипептидных векторов для каждой последовательности.
    dipeptide_seq : list of str
        Список всех возможных дипептидов, определяющий порядок и количество элементов в итоговом векторе.

    Возвращает:
    list of list of float
        Список векторов, где каждый вектор представляет дипептидное векторное представление одной последовательности.

    Пример использования:
    final_vector = getvector(list_dictionary, dipeptide_seq)
    # Возвращает список векторов для каждой последовательности.
    """
    final_vector = []  # Инициализация списка для хранения итоговых векторов

    # Перебор каждого словаря в списке
    for dic in vector_dictionary:
        vec = [0.0] * len(dipeptide_seq)
        for item in dic:
            if item in dipeptide_seq:
                index = dipeptide_seq.index(item)
                vec[index] = dic[item]

        final_vector.append(vec)

    return final_vector

In [12]:
# Hepls для извлечения векторов для транслирования модели
training_vector_dict = getlist_of_dictionary(x_train)

X_Train = getvector(training_vector_dict, dipeptide_seq)

## Данные преобразуются в массив numpy.
x = np.array(X_Train)
y = np.array(y_train)

In [13]:
# Разделим данные на обучающие и проверочные 70% и 30% соответственно.
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.3, random_state=42)

print('Разделим данные на обучающие и проверочные 70% и 30% соответственно')

print('Размер обучающих данных : ',x_train.shape)
print('Размер проверочных данных : ',x_val.shape)

Разделим данные на обучающие и проверочные 70% и 30% соответственно
Размер обучающих данных :  (27237, 780)
Размер проверочных данных :  (11673, 780)


In [14]:
"""Test data"""

## То же извлечение дипептидного вектора из тестовых данных

ID = []
test_sequence = []

with open(test_data) as f:
    for line in f:
        after_split = line.split(",")
        if after_split[0] == 'ID':
            continue;
        ID.append(after_split[0])
        test_sequence.append(after_split[1][:-1])

## То же извлечение дипептидного вектора из тестовых данных

testing_vector_dict = getlist_of_dictionary(test_sequence)
x_test = getvector(testing_vector_dict, dipeptide_seq)
x_test = np.array(x_test)

print('Размер проверочных данных : ',x_test.shape)

Размер проверочных данных :  (2000, 780)


In [15]:
"""Normalization"""

# Масштабирование данных с помощью min-max скалера
sc = MinMaxScaler()

X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [16]:
"""Feature Reduction"""

# С помощью анализа главных компонент (PCA) число признаков уменьшается с 318 до меньшего.
# Сокращение числа функций до 125
pca = PCA(n_components=125)

x_train_pca = pca.fit_transform(X_train)
x_test_pca = pca.transform(X_test)

In [17]:
# Применение RepeatedStratifiedKFold
cv_method = RepeatedStratifiedKFold(n_splits=5,
                                    n_repeats=3,
                                    random_state=999)

In [18]:
def model_training(x_train, x_test):
    """
    Предоставляет интерфейс для выбора и обучения различных моделей машинного обучения с последующей записью предсказаний в файл.

    Функция предлагает пользователю выбрать одну из предложенных моделей машинного обучения, проводит настройку параметров модели с использованием
    GridSearchCV, обучает модель на обучающем наборе данных, вычисляет точность на валидационном наборе, делает предсказания на тестовом наборе данных
    и записывает эти предсказания в файл. Поддерживаемые модели включают Случайный лес, Многослойный персептрон, Наивный Байес, XGBoost и KNN.

    Параметры:
    Нет параметров.

    Возвращает:
    Ничего не возвращает. Результат работы - файлы с предсказаниями для выбранной модели.

    Примеры использования:
    model_training() # Запуск функции приведет к интерактивному выбору модели и последующему обучению с записью предсказаний в файл.
    """

    def writeintopredfile(filename, model_pre):
        """
        Записывает предсказания модели в файл.

        Аргументы:
        filename : str
            Имя файла для сохранения предсказаний.
        model_pre : list
            Список предсказаний модели.
        """
        f = open(filename,'w')
        s = "ID,Метка\n"
        c = 0
        for i in model_pre:
            s = s + ID[c] + "," + i.__str__() + "\n"
            c += 1

        f.write(s)
        f.close()

    print('')

    print('Выберите модель: ')
    print('1. Случайный лес')
    print('2. Многослойный персептрон')
    print('3. Наивный Байес')
    print('4. XGBoost')
    print('5. KNN')

    print('')

    k = int(input('Введите номер для выбора модели: '))

    if k == 1 :

        # Настройка параметров с помощью GridSearchCV
        param_grid = {
            'bootstrap': [True],
            'max_depth': [80, 90, 100],
            'max_features': [2, 3],
            'n_estimators': [300, 400]
        }

        rf = RandomForestClassifier()
        grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                                  cv = 3, n_jobs = -1, verbose = 100)
        # Модель обучается с использованием разделенных данных
        grid_search.fit(x_train, y_train)
        # Из заданных параметров выводятся лучшие.
        print('Лучшие параметры: ', grid_search.best_params_)
        print('Точность на валидации: ', accuracy_score(y_val, grid_search.predict(x_val)))
        ans_RF = grid_search.predict(x_test)
        print('Предсказание на тестовых данных: ', ans_RF)
        writeintopredfile('RF_86.csv', ans_RF)

    elif k == 2 :

        # Количество итераций установлено как 100. Это было проверено с различными значениями максимальных итераций.
        mlp_gs = MLPClassifier(max_iter=100)
        # Настройка параметров с помощью GridSearchCV
        parameter_space = {
            'hidden_layer_sizes': [(10,30,10)],
            'activation': ['tanh'],
            'solver': ['adam'],
            'alpha': [0.05],
            'learning_rate': ['constant'],
        }
        # Кросс-валидация установлена как 10.
        clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, verbose = 50 ,cv=10)
        # Модель обучается с использованием разделенных данных
        clf.fit(x_train, y_train)
        # Из заданных параметров выводятся лучшие.
        print('Лучшие найденные параметры:\n', clf.best_params_)
        print('Оценка на валидации: ', accuracy_score(y_val, clf.predict(x_val)))
        # Производится предсказание на тестовых данных
        y_pred = clf.predict(x_test)
        print('Предсказание на тестовых данных: ', y_pred)
        writeintopredfile('MLP_67.csv', y_pred)

    elif k == 3 :

        from sklearn.naive_bayes import GaussianNB

        np.random.seed(999)

        nb_classifier = GaussianNB()

        params_NB = {'var_smoothing': np.logspace(0, -9, num=100)}

        gs_NB = GridSearchCV(estimator=nb_classifier,
                            param_grid=params_NB,
                            cv=cv_method,
                            verbose=100,
                            scoring='accuracy')

        gs_NB.fit(x_train, y_train)
        print('Лучшие параметры: ', gs_NB.best_params_)
        print('Лучшая оценка: ', gs_NB.best_score_)

        ans_NB= gs_NB.predict(x_test)
        print('Предсказание на тестовых данных: ', ans_NB)
        writeintopredfile('NB_65.csv', ans_NB)

    elif k == 4:

        validationPredictions = []
        trainingPredictions = []
        xgbClassifiersUsed = []

        xgboostClassifier = xgb.XGBClassifier(booster='gbtree', n_estimators=100, max_depth=6)
        xgboostClassifier.fit(x_train, y_train)

        xgbClassifiersUsed.append(xgboostClassifier)

        YBoostTrain = xgboostClassifier.predict(x_train)
        YBoostValidate = xgboostClassifier.predict(x_val)

        print("\nXGBoost Forest :-")
        print("Точность на обучающем наборе: ", accuracy_score(y_train, YBoostTrain))
        print("Точность на валидационном наборе: ", accuracy_score(y_val, YBoostValidate))

        ans_XGB = xgboostClassifier.predict(x_test)

        print('Предсказание на тестовых данных: ', ans_XGB)

        writeintopredfile('XGB_71.csv', ans_XGB)

    elif k == 5 :

        params_KNN = {'n_neighbors': [4],
                      'p': [1]}

        gs_KNN = GridSearchCV(estimator=KNeighborsClassifier(),
                              param_grid=params_KNN,
                              cv=cv_method,
                              verbose=100,  # Чем выше значение, тем больше сообщений
                              scoring='accuracy',
                              return_train_score=True)

        gs_KNN.fit(x_train, y_train)

        gs_KNN.best_params_

        accuracy_score(y_val, gs_KNN.predict(x_val))

        ans_KNN = gs_KNN.predict(x_test)

        writeintopredfile('KNN_65.csv', ans_NB)

    else:
        print('Вы ввели неверные данные')

In [19]:
model_training(x_train, x_test)


Выберите модель: 
1. Случайный лес
2. Многослойный персептрон
3. Наивный Байес
4. XGBoost
5. KNN



Введите номер для выбора модели:  1


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3; 1/12] START bootstrap=True, max_depth=80, max_features=2, n_estimators=300
[CV 3/3; 1/12] START bootstrap=True, max_depth=80, max_features=2, n_estimators=300
[CV 2/3; 1/12] START bootstrap=True, max_depth=80, max_features=2, n_estimators=300
[CV 2/3; 3/12] START bootstrap=True, max_depth=80, max_features=3, n_estimators=300
[CV 1/3; 3/12] START bootstrap=True, max_depth=80, max_features=3, n_estimators=300
[CV 3/3; 3/12] START bootstrap=True, max_depth=80, max_features=3, n_estimators=300
[CV 2/3; 4/12] START bootstrap=True, max_depth=80, max_features=3, n_estimators=400
[CV 2/3; 2/12] START bootstrap=True, max_depth=80, max_features=2, n_estimators=400
[CV 1/3; 2/12] START bootstrap=True, max_depth=80, max_features=2, n_estimators=400
[CV 3/3; 6/12] START bootstrap=True, max_depth=90, max_features=2, n_estimators=400
[CV 1/3; 4/12] START bootstrap=True, max_depth=80, max_features=3, n_estimators=400
[CV 3/3; 5/12] 